# This notebook contains code for adversarial attacks of resnet + TTRL

In [1]:
# !pip install foolbox
# !pip install -U tensorly
# !pip install -U tensorly-torch

In [2]:
import tltorch
import torch
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
import foolbox as fb

C:\Users\Sasha\anaconda3\lib\site-packages\tltorch\factorized_tensors\core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


In [ ]:
from pynvml import *
nvmlInit()
for ind in range(torch.cuda.device_count()):
    print(f'GPU {ind}: ')
    h = nvmlDeviceGetHandleByIndex(ind)
    info = nvmlDeviceGetMemoryInfo(h)
    print(f'\ttotal    : {info.total}')
    print(f'\tfree     : {info.free}')
    print(f'\tused     : {info.used}')

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [5]:
def get_cifar10_transform(train=True):
    if train:
        transform = T.Compose([
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768), inplace=True)

        ])

    else:
        transform = T.Compose([
            T.ToTensor(),
            T.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768))
        ])

    return transform

In [6]:
batch_size = 32

train_set = CIFAR10('CIFAR10', train=True, download=True,
                    transform=get_cifar10_transform(train=True))
test_set = CIFAR10('CIFAR10', train=False, download=True,
                   transform=get_cifar10_transform(train=False))
train_loader = DataLoader(train_set, batch_size=batch_size, pin_memory=True, num_workers=2, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, pin_memory=True, num_workers=2)


Extracting CIFAR10\cifar-10-python.tar.gz to CIFAR10
Files already downloaded and verified


In [7]:
def load_checkpoint(path, model, **kwargs):
    model = model.to(device)

    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch = checkpoint['epoch']
    train_loss = checkpoint['train_loss']
    test_loss = checkpoint['test_loss']
    train_accuracy = checkpoint['train_accuracy']
    test_accuracy = checkpoint['test_accuracy']

    return epoch, model, train_loss, test_loss, train_accuracy, test_accuracy

In [8]:
def eval_adv_acc(fmodel, val_dataloader, epsilons, attack, device="cuda:0", desc='Evaluating...'):
    model.to(device)
    model.eval()

    accuracy = np.zeros(len(epsilons))

    for images, labels in tqdm(val_dataloader, desc=desc):

        images = images.to(device)
        labels = labels.to(device)
        raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
    
        accuracy += (1 - is_adv.double().mean(axis=-1)).cpu().numpy()

    return accuracy / len(val_dataloader)

In [9]:
import sys
sys.path.insert(0,'../models')
from TTCL import TTCL
from conv_models import Model, ModelConv

## PGD attack

In [ ]:
# epsilons = [2 * 1e-3 / 255, 8 * 1e-3 / 255, 16 * 1e-3 / 255]
epsilons = np.linspace(0.0, 1e-1, num=5)
attack = fb.attacks.PGD()

folder = 'trained_models/'
files = ['TTCL-conv--epoch100.pt', 'TTCL-p-1-epoch100.pt', 'TTCL-p-0.9-epoch100.pt', 'TTCL-p-0.8-epoch100.pt',
         'TTCL-p-0.7-epoch100.pt', 'TTCL-p-0.6-epoch100.pt']
models = [ModelConv(), Model(device=device), Model(device=device), Model(device=device), Model(device=device)]

adv_accs = []

for i, (file, model_name) in enumerate(zip(files, models)):
    path = folder + file

    epoch, model, train_loss, test_loss, train_accuracy, test_accuracy = load_checkpoint(path, model_name)

    model = model.eval()
    fmodel = fb.PyTorchModel(model, bounds=(-3, 3))

    adv_acc = eval_adv_acc(fmodel, test_loader, epsilons, attack, device=device)
    adv_accs.append(adv_acc)

In [ ]:
print('eps', end='\t\t\t')
for eps in epsilons:
    print(f'{eps:.6f}', end='\t\t')
print()

for i, f in enumerate(files):
    print(f, end='\t')
    for j, eps in enumerate(epsilons):
        print(adv_accs[i][j], end='\t')
    print()